# 0.0 IMPORTS

## packages

In [ ]:
import os, pickle, datetime, json
import pandas as pd




pd.options.display.max_columns = 500 # this will set limit of columns to 500
pd.set_option('display.float_format', lambda x: '%.5f' % x)

%matplotlib inline
os.getcwd()

ModuleNotFoundError: No module named 'pandas'